
# CSCI 3202 — Mancala AI Project (Starter Notebook)

**Generated:** 2025-11-04 03:43  
This notebook contains:
- Mancala engine with **course-modified rules** (no extra turn on own Mancala).  
- Random, Minimax, and Alpha-Beta agents.  
- A light experiment scaffold you can scale locally (increase `N` or search depth).


In [7]:
import random, statistics, time
import pandas as pd

def run_series(p0, p1, n=50, continuation_rule=False, seed=1337):
    rng = random.Random(seed)
    wins = {0:0, 1:0, "tie":0}
    turns = []
    runtimes = []
    for k in range(n):
        s0 = rng.randrange(10**9)
        t0 = time.time()
        final_state, move_count = play_game(p0, p1, continuation_rule=continuation_rule, seed=s0)
        dt = time.time() - t0
        runtimes.append(dt)
        turns.append(move_count)
        manc0 = final_state.pits[MancalaState.mancala_index(final_state, 0)]
        manc1 = final_state.pits[MancalaState.mancala_index(final_state, 1)]
        if manc0 > manc1:
            wins[0] += 1
        elif manc1 > manc0:
            wins[1] += 1
        else:
            wins["tie"] += 1
    return wins, turns, runtimes


wins, turns, rts = run_series(random_policy, random_policy, n=100)
total = wins[0] + wins[1] + wins["tie"]

p1_win_pct = wins[0] / total * 100
p2_win_pct = wins[1] / total * 100
tie_pct = wins["tie"] / total * 100
avg_turns = statistics.mean(turns)

print("\n=== Mancala Random vs Random Results ===")
print(f"Total Games: {total}")
print(f"Player 1 Wins: {wins[0]} ({p1_win_pct:.1f}%)")
print(f"Player 2 Wins: {wins[1]} ({p2_win_pct:.1f}%)")
print(f"Ties: {wins['tie']} ({tie_pct:.1f}%)")
print(f"Average Turns per Game: {avg_turns:.2f}")

if p1_win_pct > p2_win_pct:
    print(f"\nFirst-move advantage: Player 1 wins {p1_win_pct - p2_win_pct:.1f}% more often.")
elif p2_win_pct > p1_win_pct:
    print(f"\nSecond player advantage: Player 2 wins {p2_win_pct - p1_win_pct:.1f}% more often.")
else:
    print("\nNo observable first-move advantage.")



=== Mancala Random vs Random Results ===
Total Games: 100
Player 1 Wins: 50 (50.0%)
Player 2 Wins: 45 (45.0%)
Ties: 5 (5.0%)
Average Turns per Game: 43.51

First-move advantage: Player 1 wins 5.0% more often.


## Intermediate Report Summary

**Progress:**
- Mancala game logic implemented and functional.
- Random player works and can play 100 random vs random games without error.

**Results:**
- Player 1 Wins: 50%
- Player 2 Wins: 45%
- Ties: 5%
- Average Turns per Game: 43.51

**First-Move Advantage:**
- Based on results, Player 1 shows a slight advantage of 5% (if applicable).

**Next Steps:**
- Add more player types (e.g., minimax or heuristic agent).
- Conduct larger simulations to confirm trends.